In [1]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# The Knuth-Bendix Completion Algorithm

This notebook presents the [Knuth-Bendix completion algorithm](https://en.wikipedia.org/wiki/Knuth%E2%80%93Bendix_completion_algorithm) for transforming a set of equations into a *confluent* term rewriting system.  This notebook is divided into eight sections.
- [Parsing](#Parsing)
- [Matching](#Matching)
- [Term Rewriting](#Term-Rewriting)
- [Unification](#Unification)
- [Knuth-Bendix Ordering](#The-Knuth-Bendix-Ordering)
- [Critical Pairs](#Critical-Pairs)
- [The Completion Algorithm](#The-Completion-Algorithm)
- [Examples](#Examples)

## Parsing

To begin, we need a parser that is capable of parsing terms and equations. This parser is implemented in the notebook `Parser.ipynb` and can parse equations of terms that use the binary operators `+`, `-`, `*`, `/`, `\`, `%`, and `^`.  The precedences of these operators are as follows:
1. `+` and `-` have the precedence $1$, which is the lowest precedence.
   Furthermore, they are *left-associative*.
2. `*`, `/`, `\`, `%` have the precedence $2$ and are also *left associative*.
3. `^` has the precedence $3$ and is *right associative*.

Furthermore, *function symbols* and *variables* are supported. Every string consisting of letters, digits, and underscores that does start with a letter is considered a function symbol if it is followed by an opening parenthesis.  Otherwise, it is taken to be a variable. Terms are defined inductively:
- Every variable is a term.
- If $f$ is a function symbol and $t_1$, $\cdots$, $t_n$ are terms, then $f(t_1,\cdots,t_n)$ is a term.
- If $s$ and $t$ are terms and $o$ is an operator, then $s\; o\; t$ is a term.  

The notebook `Parser.ipynb` also provides the function `to_str` for turning terms or equations into strings.  All together, the notebook provides the following functions:
- `parse_file(file_name)` parses a file containing equations between terms.
  It returns a list of the equations that have been parsed.
- `parse_equation(s)` converts the string `s` into an equation.
- `parse_term(s)` converts the string `s` into a term.
- `to_str(o)` converts an object `o` into a string.  The object `o` either is
  * a term,
  * an equation,
  * a list of equations,
  * a set of equations, or
  * a dictionary representing a *substitution*.

Terms and equations are represented as *nested tuples*.  These are defined recursively:
- a string is a *nested tuple*,
- a tuple `t` is a *nested tuple* iff `t[0]` is a string and for all 
  $i \in \{1,\cdots,\texttt{len}(t)-1\}$ we have that `t[i]` is a nested tuple.
  
The parser is implemented using the parser generator [Ply](https://www.dabeaz.com/ply/ply.html).

In [2]:
%run Parser.ipynb

Generating LALR tables


In [3]:
!cat Examples/quasigroups.eqn || type Examples\quasigroups.eqn

x * (x \ y) = y;
(x / y) * y = x;
x \ (x * y) = y;
(x * y) / y = x;

In [4]:
def test():
    t = parse_term('x * y * z')
    print(t)
    print(to_str(t))
    eq = parse_equation('i(x) * x = 1')
    print(eq)
    print(to_str(parse_file('Examples/quasigroups.eqn')))
    
test()

('*', ('*', ('$var', 'x'), ('$var', 'y')), ('$var', 'z'))
(x * y) * z
('=', ('*', ('i', ('$var', 'x')), ('$var', 'x')), ('1',))
[x * (x \ y) = y, (x / y) * y = x, x \ (x * y) = y, (x * y) / y = x]


[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Matching

The *substitution* $\sigma$ maps variables to terms.  It is represented as a dictionary.  If $t$ is a term and $\sigma$ is the substitution 
$$ \sigma = \{ x_1: s_1, \cdots, x_n:s_n \}, $$
then *applying* the substitution $\sigma$ to the term $t$ replaces the variables $x_i$ with the terms $s_i$.  The application of $\sigma$ to $t$ is written as $t\sigma$ and is defined by induction on $t$:
- $x_i\sigma := s_i$,
- $v\sigma := v$ if $v$ is a variable and $v \not\in \{x_1,\cdots,x_n\}$,
- $f(t_1,\cdots,t_n)\sigma := f(t_1\sigma, \cdots, t_n\sigma)$.
  
A term $p$ *matches* a term $t$ iff there exists a substitution $\sigma$ such that $p\sigma = t$. 

The function `is_var(t)` checks whether the term `t` is interpreted a variable. Variables are represented as nested tuples of the form `($var, name)`, where `name` is the name of the variable.

In [5]:
def is_var(t):
    return t[0] == '$var'

Given a string `x`, the function `make_var(x)` creates a variable with name `x`.

In [6]:
def make_var(x):
    return ('$var', x)

Given a term `p`, a term `t`,  and a substitution `σ`, the function `match_pattern(p, t, σ)` tries to extend the
substitution `σ` so that the equation
$$ p \sigma = t $$
is satisfied.  If this is possible, the function returns `True` and updates the substitution `σ` so that
$p \sigma = t$ holds.  Otherwise, the function returns `False`.

In [7]:
def match_pattern(pattern, term, σ):
    match pattern:
        case '$var', var:
            if var in σ:
                return σ[var] == term
            else:
                σ[var] = term           # extend σ
                return True
        case _:
            if pattern[0] == term[0] and len(pattern) == len(term):
                return all(match_pattern(pattern[i], term[i], σ) for i in range(1, len(pattern)))
            else:
                return False

In [8]:
def test():
    p = parse_term('i(x) * z')
    t = parse_term('i(i(y)) * i(y)')
    σ = {}
    match_pattern(p, t, σ)
    print(to_str(σ))
    
test()

{z: i(y), x: i(y)}


Given a term `t`, the function `find_variables(t)` computes the set of all variables occurring in `t`.  If, instead, $t$ is a list of terms or a set of terms, then `find_variables(t)` computes the set of those variables that occur in any of the terms of $t$.

In [9]:
def find_variables(t):
    if isinstance(t, set) or isinstance(t, list):
        return { var for term in t
                     for var  in find_variables(term)
               }
    if is_var(t):
        _, var = t
        return { var }
    _, *L = t
    return find_variables(L)

In [10]:
def test():
    eq = parse_equation('(x * y) * z = x * (y * z)')
    print(find_variables(eq))
    
test()

{'y', 'x', 'z'}


Given a term `t` and a substitution `σ` that is represented as a dictionary of the form
$$ \sigma = \{ x_1: s_1, \cdots, x_n:s_n \}, $$
the function `apply(t, σ)` computes the term that results from replacing the variables $x_i$ with the terms $s_i$ in `t` for all $i=1,\cdots,n$.  This term is written as $t\sigma$ and if $\sigma = \{ x_1: s_1, \cdots, x_n:s_n \}$, then $t\sigma$ is defined by induction on `t` as follows:
- $x_i\sigma := s_i$,
- $v\sigma := v$ if $v$ is a variable and $v \not\in \{x_1,\cdots,x_n\}$,
- $f(t_1,\cdots,t_m)\sigma := f(t_1\sigma, \cdots, t_m\sigma)$.

In [11]:
def apply(t, σ):
    "Apply the substitution σ to the term t."
    if is_var(t):
        _, var = t
        if var in σ:
            return σ[var]
        else:
            return t
    else: 
        f, *Ts = t
        return (f,) + tuple(apply(s, σ) for s in Ts)

In [12]:
def test():
    p = parse_term('i(x) * x')
    t = parse_term('i(i(y)) * i(y)')
    σ = {}
    match_pattern(p, t, σ)
    print(f'apply({to_str(p)}, {to_str(σ)}) = {to_str(apply(p, σ))}')

test()

apply(i(x) * x, {x: i(y)}) = i(i(y)) * i(y)


Given a set of terms or equations `Ts` and a substitution `σ`, the function `apply_set(Ts, σ)` applies the substitution `σ` to all elements in `Ts`.

In [13]:
def apply_set(Ts, σ):
    return { apply(t, σ) for t in Ts }

If  $\sigma = \big\{ x_1 \mapsto s_1, \cdots, x_m \mapsto s_m \big\}$ and
$\tau = \big\{ y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big\}$ 
are two substitutions that are *non-overlapping*, i.e. such that $\{x_1,\cdots, x_m\} \cap \{y_1,\cdots,y_n\} = \{\}$ holds,
then we define the *composition* $\sigma\tau$ of $\sigma$ and $\tau$ as follows:
$$\sigma\tau := \big\{ x_1 \mapsto s_1\tau, \cdots, x_m \mapsto s_m\tau,\; y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big\}$$
This definition implies that the following *associative law* is valid:
$$ s(\sigma\tau) = (s\sigma)\tau $$
The function $\texttt{compose}(\sigma, \tau)$ takes two non-overlapping substitutions and computes their *composition* $\sigma\tau$.

In [14]:
def compose(σ, τ):
    Result = { x: apply(s, τ) for (x, s) in σ.items() }
    Result.update(τ)
    return Result

In [15]:
def test():
    t1 = parse_term('i(y)')
    t2 = parse_term('a * b')
    t3 = parse_term('i(b)')
    σ = { 'x': t1 }
    τ = { 'y': t2, 'z': t3  }
    print(f'compose({to_str(σ)}, {to_str(τ)}) = {to_str(compose(σ, τ))}')
    
test()

compose({x: i(y)}, {y: a * b, z: i(b)}) = {x: i(a * b), y: a * b, z: i(b)}


[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Term Rewriting

In [16]:
from string import ascii_lowercase
ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

Given a term `s` and a set of variables `V`, the function `rename_variables(s, V)` renames the variables in `s` so that they differ from the variables in the set `V`.  This will only work if the number of variables occurring in `V` times two is less than the number of letters in the latin alphabet, i.e. less than 26.  Therefore, the set `V` must have at most 13 variables.  For our examples, this is not a restriction.

In [17]:
def rename_variables(s, Vars):
    assert len(Vars) <= 13, f'Error: too many variables in {Vars}.'
    NewVars = set(ascii_lowercase) - Vars
    NewVars = sorted(list(NewVars))
    σ       = { x: make_var(NewVars[i]) for (i, x) in enumerate(Vars) }
    return apply(s, σ)

In [18]:
def test():
    t = parse_equation('x * y * z = x * (y * z)')
    V = find_variables(t)
    print(f'rename_variables({to_str(t)}, {V}) = {to_str(rename_variables(t, V))}')
    
test()

rename_variables((x * y) * z = x * (y * z), {'y', 'x', 'z'}) = (b * a) * c = b * (a * c)


The function `simplify_step(t, E)` takes two arguments:
- `t` is a term,
- `E` is a set of equations of the form `('=', l, r)`.

The function tries to find an equation `l = r` in `E` and a subterm `s` in the term `t` such that the left hand side `l` of the equation matches the subterm `s` using some substitution $\sigma$, i.e. we have $s = l\sigma$.  Then the term `t` is simplified by replacing the subterm `s` in `t` by $r\sigma$.  More formally, if `u` is the position of `s` in `t`, i.e. `t/u = s` then `t` is simplified into the term 
$$ t = t[u \mapsto l\sigma] \rightarrow_{\{l=r\}} t[u \mapsto r\sigma]. $$
If an appropriate subterm `s` is found, the simplified term is returned.  Otherwise, the function returns `None`.  

If multiple subterms of `t` can simplified, then the function `simplify_step(t, E)` simplifies all subterms.

In [19]:
def simplify_step(t, Equations):
    if is_var(t):
        return None      # variables can't be simplified
    for eq in Equations:
        _, lhs, rhs = eq
        σ = {}
        if match_pattern(lhs, t, σ):
            return apply(rhs, σ)
    f, *args   = t
    simpleArgs = []
    change     = False
    for arg in args:
        simple = simplify_step(arg, Equations)
        if simple != None:
            simpleArgs += [simple]
            change = True
        else:
            simpleArgs += [arg]
    if change:
        return (f,) + tuple(simpleArgs)
    return None

In [20]:
def test():
    E = { parse_equation('(x * y) * z = x * (y * z)') }
    t = parse_term('(a * b) * i(b)')
    print(f'simplify_step({to_str(t)}, {to_str(E)}) = {to_str(simplify_step(t, E))}')
    
test()

simplify_step((a * b) * i(b), {(x * y) * z = x * (y * z)}) = a * (b * i(b))


The function `normal_form(t, E)` takes a term `t` and a list (or set) of equations `E` and tries to simplify the term `t` as much as possible using the equations from `E`.  

In the implementation, we have to be careful to rename the variables occurring in `E` so that they are different from the variables occurring in `t`.  **Furthermore, we have to take care that we don't identify different variables in `E` by accident.**  Therefore, we rename the variables in `E` so that they are both different from the variables in `t` and from the old variables occurring in `E`.

In [21]:
def normal_form(t, E):
    Vars = find_variables(t) | find_variables(E)
    NewE = []
    for eq in E:
        NewE += [ rename_variables(eq, Vars) ]   
    while True:
        s = simplify_step(t, NewE)
        if s == None:
            return t
        t = s

In [22]:
!cat Examples/group-theory-1.eqn || type Examples\group-theory-1.eqn

      1 * x = x;
   i(x) * x = 1;
(x * y) * z = x * (y * z);

In [23]:
def test():
    E  = parse_file('Examples/group-theory-1.eqn')
    t  = parse_term('1 * (b * i(a)) * a')
    print(f'E = {to_str(E)}')
    print(f'normal_form({to_str(t)}, E) = {to_str(normal_form(t, E))}')
    
test()

E = [1 * x = x, i(x) * x = 1, (x * y) * z = x * (y * z)]
normal_form((1 * (b * i(a))) * a, E) = b * 1


[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Unification

In this section, we implement the [unification](https://en.wikipedia.org/wiki/Unification_(computer_science)) algorithm of Martelli and Montanari.

Given a variable name `x` and a term `t`, the function `occurs(x, t)` checks whether `x` occurs in `t`.

In [24]:
def occurs(x, t):
    if is_var(t):
        _, var = t
        return x == var
    return any(occurs(x, arg) for arg in t[1:])

The algorithm implemented below takes a pair `(E, σ)` as its input.  Here `E` is a set of *syntactical equations* that need to be solved and `σ` is a substitution that is initially empty.  The pair `(E, σ)` is then transformed using the rules of *Martelli and Montanari*.  The transformation is successful if the pair `(E, σ)` can be transformed into a pair of the form `({}, μ)`.  Then `μ` is the *solution* to the system of equations `E` and hence `μ` is a *most general unifier* of `E`.

The rules that can be used to solve a system of *syntactical equations* are as follows:
   - If $y\in\mathcal{V}$ is a variable that does **not** occur in the term $t$,
     then we perform the following reduction: 
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E[y \mapsto t], \sigma\big[ y \mapsto t \big] \Big\rangle 
     $$     
   - If the variable $y$ occurs in the term $t$ and $y$ is different from $t$, then the system of 
     syntactical equations
     $E \cup \big\{ y \doteq t \big\}$ is not solvable:
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle\;\leadsto\; \texttt{None} \quad
        \mbox{if $y \in \textrm{Var}(t)$ and $y \not=t$.}$$
   - If $y\in\mathcal{V}$ is a variable and $t$ is no variable, then we use the following rule:
     $$ \Big\langle E \cup \big\{ t \doteq y \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle.
     $$   
   - Trivial syntactical equations of variables can be dropped:
     $$ \Big\langle E \cup \big\{ x \doteq x \big\}, \sigma \Big\rangle \quad\leadsto \quad
         \Big\langle E, \sigma \Big\rangle.
     $$   
   - If $f$ is an $n$-ary function symbol, then we have: 
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_n) \doteq f(t_1,\cdots,t_n) \big\}, \sigma \Big\rangle 
         \;\leadsto\; 
         \Big\langle E \cup \big\{ s_1 \doteq t_1, \cdots, s_n \doteq t_n\}, \sigma \Big\rangle.
     $$   
   - The system of syntactical equations $E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\}$
     has **no** solution if the function symbols $f$ and $g$ are different:
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\},
      \sigma \Big\rangle \;\leadsto\; \texttt{None} \qquad \mbox{if $f \not= g$}.
     $$

Given two terms $s$ and $t$, the function $\texttt{unify}(s, t)$ computes the <em style="color:blue;">most general unifier</em> of $s$ and $t$.

In [25]:
def unify(s, t):
    return solve({('≐', s, t)}, {})

Given a set of <em style="color:blue;">syntactical equations</em> $E$ and a substitution $\sigma$, the function $\texttt{solve}(E, \sigma)$ applies the rules of Martelli and Montanari to solve $E$.

In [26]:
def solve(E, σ):
    while E != set():
        _, s, t = E.pop()
        if s == t: # remove trivial equations
            continue
        if is_var(s):
            _, x = s
            if occurs(x, t):
                return None
            else: # set x to t
                E = apply_set(E, { x: t })
                σ = compose(σ, { x: t })
        elif is_var(t):
            E.add(('≐', t, s))
        else:
            f    , g     = s[0]      , t[0]
            sArgs, tArgs = s[1:]     , t[1:]
            m    , n     = len(sArgs), len(tArgs)
            if f != g or m != n:
                return None
            else:
                E |= { ('≐', sArgs[i], tArgs[i]) for i in range(m) }
    return σ

In [27]:
def test():
    s = parse_term('x * i(x) * (y * z)')
    t = parse_term('a * i(1) * b')
    print(f'unify({to_str(s)}, {to_str(t)}) = {to_str(unify(s, t))}')
    
test()

unify((x * i(x)) * (y * z), (a * i(1)) * b) = {x: 1, a: 1, b: y * z}


[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## The Knuth-Bendix Ordering

In order to turn an equation $s = t$ into a rewrite rule, we have to check whether the term $s$ is *more complex* than the term $t$, so that $s$ should be simplified to $t$, or whether $t$ is *more complex* than $s$ and we should rewrite $t$ into $s$.  To this end, we implement the *Knuth-Bendix ordering*, which is a method to compare terms.

Given a term `t` and a variable name `x`, the function `count(t, x)` computes the number of times that `x` occurs in `t`.

In [28]:
def count(t, x):
    match t:
        case '$var', y:
            return 1 if x == y else 0
        case _, *Ts:
            return sum(count(arg, x) for arg in Ts)

In [29]:
def test():
    t = parse_term('x * (i(x) * y)')
    print(f'count({to_str(t)}, "x") = {count(t, "x")}')
    
test()

count(x * (i(x) * y), "x") = 2


In order to define the *Knuth-Bendix ordering* on terms, three prerequisites need to be satisfied:
1. We need to assign a *weight* $w(f)$ to every function symbol $f$.  These weights are 
   natural numbers.  There must be at most one function symbol $g$ such that $w(g) = 0$.
   Furthermore, if $w(g) = 0$, then $g$ has to be unary.
   
   We define the weights via the dictionary `Weight`, i.e. we have $w(f) = \texttt{Weight}[f]$.
2. We need to define a *strict order* $<$ on the set of function symbols.

   This ordering is implemented via the dictionary `Ordering`.  We define
   $$ f < g \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\; \texttt{Ordering}[f] < \texttt{Ordering}[f]. $$
3. The order $<$ on the function symbols has to be *admissible* with respect to the weight function $w$, i.e. the following
   condition needs to be satisfied:
   $$ w(f) = 0 \rightarrow \forall g:  \bigl(g \not=f \rightarrow g < f\bigr). $$
   To put this in words: If the function symbol $f$ has a weight of $0$, then 
   all other function symbols $g$ have to be smaller than $f$ w.r.t. the strict order $<$.
   Note that this implies that there can be at most    one function symbol with $f$ such that $w(f) = 0$. 
   This function symbol $f$ is then the maximum w.r.t. the order $<$.
   
   Below, for efficiency reasons, the function `max_fct` returns the function symbol $f$ that is maximal w.r.t. the strict order $<$.

In [30]:
WEIGHT   = { '1': 1, '*': 1, '/': 1, '\\': 1, 'i': 0 }
ORDERING = { '1': 0, '*': 1, '/': 2, '\\': 3, 'i': 5 }
max_fct  = lambda: 'i' 

Given a term `t` the function `weight(t)` computes the *weight* $w(t)$, where $w(t)$ is defined by induction on $t$:
- $w(x) := 1$ for all variables $x$,
- $w\bigl(f(t_1,\cdots,t_n)\bigr) := \texttt{Weight}[f] + \sum\limits_{i=1}^n w(t_i)$.

In [31]:
def weight(t):
    match t:
        case '$var', _:
            return 1
        case f, *Ts:
            return WEIGHT[f] + sum(weight(arg) for arg in Ts)

In [32]:
def test():
    t = parse_term('x * (i(x) * 1)')
    print(f'weight({to_str(t)}) = {weight(t)}')
    
test()    

weight(x * (i(x) * 1)) = 5


Given a term `s` and a term `t`, the function `is_tower(s, t)` returns `True` iff the following is true:
$$ \exists n\in\mathbb{N}:\bigl( n > 0 \wedge t = f^{n}(s) \wedge f = \texttt{max_fct}()\bigr). $$  
Here the expression $f^n(s)$ is the $n$-fold application of $f$ to $s$, e.g. we have $f^1(s) = f(s)$, $f^2(s) = f(f(s))$, and in general $f^{n+1}(s) = f\bigl(f^{n}(s)\bigr)$.

In [33]:
def is_tower(s, t):
    if len(t) != 2:   # f is not unary
        return False
    f, t1 = t
    if f != max_fct():
        return False
    if t1 == s:
        return True
    return is_tower(s, t1)

In [34]:
def test():
    s = parse_term('i(a)')
    t = parse_term('i(i(a))')
    print(f'is_tower({to_str(s)}, {to_str(t)}) = {is_tower(s, t)}')
    
test()    

is_tower(i(a), i(i(a))) = True


The *Knuth-Bendix order* $s \prec_{\textrm{kbo}} t$ is defined for terms $s$ and $t$.  We have $s \prec_{\textrm{kbo}} t$ iff one of the following two conditions hold:
1. $w(s) < w(t)$ and $\texttt{count}(s, x) \leq \texttt{count}(t, x)$ for all variables $x$ occurring in $s$ .
2. $w(s) = w(t)$, $\texttt{count}(s, x) \leq \texttt{count}(t, x)$ for all variables $x$ occurring in $s$, and
   one of the following subconditions holds:
   * $t = f^n(s)$ where $n \geq 1$ and $f$ is the maximum w.r.t. the order $<$ on function symbols,
     i.e. we have $f = \texttt{max_fct}()$.
   * $s = f(s_1,\cdots,s_m)$, $t=g(t_1,\cdots,t_n)$, and $f<g$.
   * $s = f(s_1,\cdots,s_m)$, $t=f(t_1,\cdots,t_m)$, and
     $[s_1,\cdots,s_m] \prec_{\textrm{lex}} [t_1,\cdots,t_m]$.
     
     Here, $\prec_{\textrm{lex}}$ denotes the *lexicographic extension* of the ordering $\prec_{\textrm{kbo}}$ to
     lists of terms.  It is defined as follows:
     $$ [x] + R_1 \prec_{\textrm{lex}} [y] + R_2 \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\;
      x \prec_{\textrm{kbo}} y \,\vee\, \bigl(x = y \wedge R_1 \prec_{\textrm{lex}} R_2\bigr)
     $$

Given two terms `s` and `t` the function `is_simpler(s, t)` returns `True` if $s \prec_{\textrm{kbo}} t$.

In [35]:
def is_simpler(s, t):
    if is_var(t):
        return False
    if is_var(s):
        _, x = s
        return occurs(x, t)
    Vs = find_variables(s)
    for x in Vs:
        if count(t, x) < count(s, x):
            return False
    ws = weight(s)
    wt = weight(t)
    if ws < wt:
        return True    
    if ws > wt:
        return False
    # ws == wt
    if is_tower(s, t):
        return True
    f, *Ss = s
    g, *Ts = t
    if ORDERING[f] < ORDERING[g]:
        return True
    if ORDERING[f] > ORDERING[g]:
        return False
    return is_simpler_list(Ss, Ts)

Given two lists `S` and `T` of terms, the function `is_simpler_list(S, T)` checks whether `S` is lexicographically simpler than `T` if the elements of `S` and `T` are compared with the *Knuth-Bendix ordering* $\prec_{\textrm{kbo}}$.  It is assumed that `S` and `T` have the same length.

In [36]:
def is_simpler_list(S, T):
    if S == [] == T:
        return False
    if is_simpler(S[0], T[0]):
        return True
    if S[0] == T[0]:
        return is_simpler_list(S[1:], T[1:])
    return False

In [37]:
def test():
    #l = parse_term('(x * y) * z')
    #r = parse_term('x * (y * z)')
    l = parse_term('i(a)')
    r = parse_term('i(i(a))')
    print(f'is_simpler({to_str(r)}, {to_str(l)}) = {is_simpler(r, l)}')
    print(f'is_simpler({to_str(l)}, {to_str(r)}) = {is_simpler(l, r)}')
    
test()

is_simpler(i(i(a)), i(a)) = False
is_simpler(i(a), i(i(a))) = True


We define the class `OrderException` to be able to deal with equations that can't be ordered into a rewrite rule. 

In [38]:
class OrderException(Exception):
    pass

Given an equation `eq` and an `Ordering` of the function symbols occurring `eq`, the function `order_equation` orders the equation `eq` with respect to the *Knuth-Bendix ordering*, i.e. in the ordered equation, the right hand side is simpler than the left hand side.  If the left hand side and the right hand side are incomparable, the function raises an `OrderException`.

In [39]:
def order_equation(eq):
    _, s, t = eq
    if is_simpler(t, s):
        return ('=', s, t)
    elif is_simpler(s, t):
        return ('=', t, s)
    else:
        Msg = f'Knuth-Bendix algorithm failed: Could not order {to_str(s)} = {to_str(t)}'
        raise OrderException(Msg)

In [40]:
def test():
    equation = 'i(i(a)) = i(i(i(i(a))))'
    eq = parse_equation(equation)
    print(f'order_equation({to_str(eq)}) = {to_str(order_equation(eq))}')

test()

order_equation(i(i(a)) = i(i(i(i(a))))) = i(i(i(i(a)))) = i(i(a))


[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Critical Pairs

The central notion of the Knuth-Bendix algorithm is the notion of a *critical pair*.  

Given two equations `lhs1 = rhs1` and `lhs2 = rhs2`, a pair of terms `(s, t)` is a critical pair of these equations if we have the following:
- `u` is a non-trivial position in `lhs1`, i.e. `lhs1/u` is not a variable,
- The subterm `lhs1/u` is unifiable with `lhs2`, i.e.
  $$\mu = \texttt{mgu}(\texttt{lhs}_1 / \texttt{u}, \texttt{lhs}_2) \not= \texttt{None},$$
- $s = \texttt{lhs}_1\mu[\texttt{u} \mapsto \texttt{rhs}_2\mu]$ and $t = \texttt{rhs}_1\mu$. 

The idea is then that the term $\texttt{lhs1}\mu$ can be rewritten into different ways:
- $\texttt{lhs1}\mu \rightarrow \texttt{rhs1}\mu = t$,
- $\texttt{lhs1}\mu \rightarrow \texttt{lhs}_1\mu[\texttt{u} \mapsto \texttt{rhs}_2\mu] = s$.

The function `critical_pairs` implemented in this section computes the critical pairs between two rewrite rules. 

Given a term `t`, the function `non_triv_positions` computes the set $\mathcal{P}os(t)$ of all *positions* in `t` that do not point to variables.  Such positions are called *non-trivial positions*.  Given a term `t`, the set $\mathcal{P}os(t)$ of all positions in $t$ is defined by induction on `t`. 
1. $\mathcal{P}os(v) := \bigl\{()\bigr\} \quad \mbox{if $v$ is a variable} $
2. $\mathcal{P}os\bigl(f(t_0,\cdots,t_{n-1})\bigr) := 
     \bigl\{()\bigr\} \cup 
     \bigl\{ (i,) + u \mid i \in\{0,\cdots,n-1\} \wedge u \in \mathcal{P}os(t_i) \bigr\}
   $
   
Note that since we are programming in Python, positions are zero-based.  Given a position $v$ in a term $t$, we define $t/v$ as the *subterm of $t$ at position $v$* by induction on $t$:
1. $t/() := t$,
2. $f(t_0,\cdots,t_{n-1})/u := t_{u\texttt{[0]}}/u\texttt{[1:]}$.

Given a term $s$, a term $t$, and a position $u \in \mathcal{P}os(t)$, we also define the *replacement of the subterm at position $u$ by $t$*, written $s[u \mapsto t]$ by induction on $u$:
1. $s\bigl[() \mapsto t\bigr] := t$.
2. $f(s_0,\cdots,s_{n-1})\bigl[\bigl((i,) + u\bigr) \mapsto t\bigr] := f\bigl(s_0,\cdots,s_i[u \mapsto t],\cdots,s_{n-1}\bigr)$.

In [41]:
def non_triv_positions(t):
    if is_var(t):
        return set()
    _, *args = t
    Result = { () }
    for i, arg in enumerate(args):
        Result |= { (i,) + a for a in non_triv_positions(arg) }
    return Result

In [42]:
def test():
    t = parse_term('x * i(x) * 1')
    print(f'non_triv_positions({to_str(t)}) = {non_triv_positions(t)}')
    
test()

non_triv_positions((x * i(x)) * 1) = {(0, 1), (1,), (), (0,)}


Given a term `t` and a position `u` in `t`, the function `subterm(t, u)` extracts the subterm that is located at position `u`, i.e. it computes `t/u`. The position `u` is zero-based.

In [43]:
def subterm(t, u):
    if len(u) == 0:
        return t
    _, *args = t
    i, *ur   = u
    return subterm(args[i], ur)

In [44]:
def test():
    t = parse_term('(x * i(x)) * 1')
    print(f'subterm({to_str(t)}, (0,1)) = {to_str(subterm(t, (0,1)))}')
    
test()

subterm((x * i(x)) * 1, (0,1)) = i(x)


Given a term `t`, a position `u` in `t` and a term `s`, the function `replace_at(t, u, s)` replaces the subterm at position `u` with `t`.  The position `u` uses zero-based indexing.  Hence it returns the term
$$ t[u \mapsto s]. $$

In [45]:
def replace_at(t, u, s):
    if len(u) == 0:
        return s
    i, *ur   = u
    f, *Args = t
    NewArgs  = []
    for j, arg in enumerate(Args):
        if j == i:
            NewArgs.append(replace_at(arg, ur, s))
        else:
            NewArgs.append(arg)
    return (f,) + tuple(NewArgs)

In [46]:
def test():
    t = parse_term('(x * i(x)) * 1')
    s = parse_term('a * b') 
    print(f'replace_at({to_str(t)}, (0,1), {to_str(s)}) = {to_str(replace_at(t, (0,1), s))}')

test()

replace_at((x * i(x)) * 1, (0,1), a * b) = (x * (a * b)) * 1


Given two equations `eq1` and `eq2`, the function `critical_pairs(eq1, eq2)` computes the set of all *critical pairs* between these equations.  A pair of terms `(s, t)` is a critical pair of `eq1` and `eq2` if we have
- `eq1` has the form `lhs1 = rhs1`,
- `eq2` has the form `lhs2 = rhs2`,
- `u` is a non-trivial position in `lhs1`,
- $\mu = \texttt{mgu}(\texttt{lhs}_1/u, \texttt{lhs}_2) \not= \texttt{None}$,
- $s = \texttt{lhs}_1\mu[u \leftarrow \texttt{rhs}_2\mu]$ and $t = \texttt{rhs}_1\mu$. 

In [47]:
def critical_pairs(eq1, eq2):
    Vars = find_variables(eq1) | find_variables(eq2)
    eq2  = rename_variables(eq2, Vars) 
    _, lhs1, rhs1 = eq1
    _, lhs2, rhs2 = eq2
    Result = set()
    Positions = non_triv_positions(lhs1)
    for u in Positions:
        𝜇 = unify(subterm(lhs1, u), lhs2)
        if 𝜇 != None:
            lhs1_new = apply(replace_at(lhs1, u, rhs2), 𝜇)
            rhs1_new = apply(rhs1, 𝜇)
            Result.add( (('=', lhs1_new, rhs1_new), eq1, eq2))
    return Result

In [48]:
def test():
    eq1 = parse_equation('(x * y) * z = x * (y * z)')
    eq2 = parse_equation('i(x) * x = 1')
    for ((_, s, t), _, _) in critical_pairs(eq1, eq2):
        print(f'critical_pairs({to_str(eq1)}, {to_str(eq2)}) = ' + '{' + f'{to_str(s)} = {to_str(t)}' + '}')
        
test()

critical_pairs((x * y) * z = x * (y * z), i(x) * x = 1) = {1 * z = i(b) * (b * z)}


[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## The Completion Algorithm

Given a set of `RewriteRules` and a newly derived rewrite `rule`, the function `simplify_rules(RewriteRules, rule)` adds `rule` to the set `RewriteRules`.  When the function returns, every equation in the set `RewriteRules` is in normal form with respect to all other equations in `RewriteRules`. 

In [49]:
def simplify_rules(RewriteRules, rule):
    UnusedRules = [ rule ]
    while UnusedRules != []:
        UnchangedRules = set()
        r = UnusedRules.pop()
        for eq in RewriteRules:
            simple = normal_form(eq, { r })
            if simple != eq:
                simple = normal_form(simple, RewriteRules | { r })
                if simple[1] != simple[2]:
                    simple = order_equation(simple)
                    UnusedRules.append(simple)
                    print('simplified:')
                    print(f'old:     {to_str(eq)}')
                    print(f'new:     {to_str(simple)}')
                else:
                    print(f'removed: {to_str(eq)}')
            else:
                UnchangedRules.add(eq)
        RewriteRules = UnchangedRules | { r }
    return RewriteRules

The function `print_equations` prints the set of `Equations` one by one and numbers them.

In [50]:
def print_equations(Equations):
    cnt = 1
    for _, l, r in Equations:
        print(f'{cnt}. {to_str(l)} = {to_str(r)}')
        cnt += 1

Given an equation `eq` of the form `eq = ('=', lhs, rhs)`, the function `complexity(eq)` computes a measure of complexity for the given equation.   This measure of complexity is the length of the string that represents the equation.  This measure of complexity is later used to choose between equations: Less complex equations are more interesting and should be considered first when computing critical pairs.

In [51]:
def complexity(eq):
    return len(to_str(eq))

Given a set of equations `RewriteRules` and a single rewrite rule `eq`, the function `all_critical_pairs(RewriteRules, eq)` computes the set of all *critical pairs* that can be build by building critical pairs with an equation from `RewriteRules` and the equation `eq`.  It is assumed that `eq` is already an element of `RewriteRules`.  

In [52]:
def all_critical_pairs(RewriteRules, eq):
    Result = set()
    for eq1 in RewriteRules:
        Result |= { cp for cp in critical_pairs(eq1, eq) }
        Result |= { cp for cp in critical_pairs(eq, eq1) }
    return Result

The module `heapq` provides heap-based priority queues, which are implemented as lists.

In [53]:
import heapq as hq

Given a file name that contains a set of equations and a dictionary encoding an ordering of the function symbols, the function `knuth_bendix_algorithm` implements the *Knuth-Bendix algorithm*:
1. The equations read from the file are oriented into rewrite rules.
2. These oriented equations are pushed onto the priority queue `EquationQueue` according to their complexity.
3. The set `RewriteRules` is initialized as the empty set.  The idea is that all critical pairs between
   equations in `RewriteRules` have already been computed and that the resulting new equations have been added
   to the priority queue `EquationQueue`. 
4. As long as the priority queue `EquationQueue` is not empty, the least complex equation `eq` is removed from the 
   priority queue and simplified using the known `RewriteRules`.  
5. If the simplified version of `eq` is not trivial, all critical pairs between `eq` and the 
   existing `RewriteRules` are computed.  The resulting equations are pushed onto the priority queue `EquationQueue`.
6. When no new critical pairs can be found, the set of `RewriteRules` is returned.
   This set is then guaranteed to be a *confluent* set of rewrite rules.

In [54]:
def knuth_bendix_algorithm(file):
    Equations    = set()
    Axioms       = set(parse_file(file))
    RewriteRules = set()
    try:
        for eq in Axioms:
            ordered_eq = order_equation(eq)
            Equations.add(ordered_eq)
            print(f'given:   {to_str(ordered_eq)}')
        EquationQueue = []
        for eq in Equations:
            hq.heappush(EquationQueue, (complexity(eq), eq))
        while EquationQueue != []:
            _, eq = hq.heappop(EquationQueue)
            eq = normal_form(eq, RewriteRules)
            if eq[1] != eq[2]:
                lr = order_equation(eq)
                print(f'added:   {to_str(lr)}')
                Pairs = all_critical_pairs(RewriteRules | { lr }, lr)
                for eq, r1, r2 in Pairs:
                    new_eq = normal_form(eq, RewriteRules)
                    if new_eq[1] != new_eq[2]:
                        print(f'found:   {to_str(eq)} from {to_str(r1)}, {to_str(r2)}')
                        hq.heappush(EquationQueue, (complexity(new_eq), new_eq))
                RewriteRules = simplify_rules(RewriteRules, lr)
    except OrderException as e:
        print(e)
    print()
    print_equations(RewriteRules)
    return RewriteRules

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Examples

In this section we present a number of examples where the *Knuth-Bendix completion* algorithm is able to produce a confluent system of equations.  In detail, we discuss the following examples:
1. [Group Theory](#Group-Theory)
2. [Central Groupoids](#Central-Groupoids)
3. [Quasigroups](#Quasigroups)
4. [Quasigroups with Idempotence](#Quasigroups-with-Idempotence)
5. [Quasigroups with Unipotence](#Quasigroups-with-Unipotence)
6. [Loops](#Loops)

### Group Theory

A structure $\mathcal{G} = \langle G, 1, *, i \rangle$ is a [group](https://en.wikipedia.org/wiki/Group_(mathematics)) iff
1. $G$ is a set.
2. $1 \in G$,

   where $1$ is called the *left-neutral element*.
3. $*: G \times G \rightarrow G$,

   where $*$ is called the *multiplication* of $\mathcal{G}$.
4. $i: G \rightarrow G$,

   where for any $x \in G$ the element $i(x)$ is called the *left-inverse* of $x$.
5. The following equations hold for all $x,y,z \in G$:
   * $1 * x = x$, i.e. $1$ is a *left-neutral element*.
   * $i(x) * x = 1$, i.e. $i(x)$ is a *left-inverse* of $x$.
   * $(x * y) * z = x * (y * z)$, i.e. the multiplication is *associative*.

A typical example of a group is the set of invertible $n \times n$ matrices.

Given the axioms defining a group, the *Knuth-Bendix completion* algorithm is able to prove the following:
1. The *left neutral element* is also a *right neutral element*, we have:
   $$ x * 1 = x \quad \mbox{for all $x\in G$.} $$
2. The *left inverse* is also a *right inverse*, we have:
   $$ x * i(x) = 1 \quad \mbox{for all $x\in G$.} $$
3. The operations $i$ and $*$ commute as follows:
   $$ i(x * y) = i(y) * i(x) \quad \mbox{for all $x,y\in G$.}$$

In [55]:
!cat Examples/group-theory-1.eqn || type Examples\group-theory-1.eqn

      1 * x = x;
   i(x) * x = 1;
(x * y) * z = x * (y * z);

In [56]:
%%time
Rules = knuth_bendix_algorithm('Examples/group-theory-1.eqn')

given:   (x * y) * z = x * (y * z)
given:   i(x) * x = 1
given:   1 * x = x
added:   1 * x = x
added:   i(x) * x = 1
added:   (x * y) * z = x * (y * z)
found:   1 * z = i(b) * (b * z) from (x * y) * z = x * (y * z), i(b) * b = 1
found:   (b * (a * c)) * z = (b * a) * (c * z) from (x * y) * z = x * (y * z), (b * a) * c = b * (a * c)
added:   i(b) * (b * z) = z
found:   i(i(a)) * c = a * c from i(b) * (b * z) = z, i(a) * (a * c) = c
found:   d * z = i(e) * ((e * d) * z) from (x * y) * z = x * (y * z), i(e) * (e * d) = d
found:   i(c * a) * (c * (a * d)) = d from i(b) * (b * z) = z, (c * a) * d = c * (a * d)
found:   i(i(a)) * 1 = a from i(b) * (b * z) = z, i(a) * a = 1
found:   i(1) * a = a from i(b) * (b * z) = z, 1 * a = a
added:   i(1) * a = a
found:   i(i(1)) * c = c from i(b) * (b * z) = z, i(1) * c = c
found:   b * z = i(1) * (b * z) from (x * y) * z = x * (y * z), i(1) * b = b
added:   i(i(a)) * 1 = a
found:   i(i(i(c))) * c = 1 from i(b) * (b * z) = z, i(i(c)) * 1 = c
found:   b 

It is natural to ask whether the axiom describing the *left neutral element* and the axiom describing the *left inverse* can be replaced by corresponding axioms that require $1$ to be a *right neutral element* and $i(x)$ to be a *right inverse*.  The *Knuth-Bendix completion* algorithm shows that this is indeed the case.

In [57]:
!cat Examples/group-theory-2.eqn || type Examples\group-theory-2.eqn

      x * 1 = x;
   x * i(x) = 1;
(x * y) * z = x * (y * z);

In [58]:
%%time
Rules = knuth_bendix_algorithm('Examples/group-theory-2.eqn')

given:   x * 1 = x
given:   (x * y) * z = x * (y * z)
given:   x * i(x) = 1
added:   x * 1 = x
added:   x * i(x) = 1
added:   (x * y) * z = x * (y * z)
found:   1 = x * (y * i(x * y)) from (x * y) * z = x * (y * z), b * i(b) = 1
found:   b * (a * i(b * a)) = 1 from x * i(x) = 1, (b * a) * c = b * (a * c)
found:   b * z = b * (1 * z) from (x * y) * z = x * (y * z), b * 1 = b
found:   1 * z = b * (i(b) * z) from (x * y) * z = x * (y * z), b * i(b) = 1
found:   (b * (a * c)) * z = (b * a) * (c * z) from (x * y) * z = x * (y * z), (b * a) * c = b * (a * c)
added:   b * (1 * z) = b * z
found:   c * (a * (1 * z)) = (c * a) * z from b * (1 * z) = b * z, (c * a) * d = c * (a * d)
found:   b * 1 = b * i(1) from b * (1 * z) = b * z, a * i(a) = 1
found:   (x * y) * d = x * (y * (1 * d)) from (x * y) * z = x * (y * z), e * (1 * d) = e * d
found:   (e * d) * z = e * ((1 * d) * z) from (x * y) * z = x * (y * z), e * (1 * d) = e * d
added:   b * i(1) = b
found:   x * y = x * (y * i(1)) from (x * y) *

#### LR Systems

Next, it is natural to ask what happens if we have a *left neutral element* and a *right inverse*.  Algebraic Structures of this kind are called *LR systems*.  The *Knuth-Bendix completion algorithm* shows that, in general, *LR systems* are different from groups.

In [59]:
!cat Examples/lr-system.eqn || type Examples\lr-system.eqn

      1 * x = x;
   x * i(x) = 1;
(x * y) * z = x * (y * z);

In [60]:
%%time
Rules = knuth_bendix_algorithm('Examples/lr-system.eqn')

given:   (x * y) * z = x * (y * z)
given:   x * i(x) = 1
given:   1 * x = x
added:   1 * x = x
added:   x * i(x) = 1
found:   i(1) = 1 from x * i(x) = 1, 1 * a = a
found:   1 = i(1) from 1 * x = x, a * i(a) = 1
added:   i(1) = 1
added:   (x * y) * z = x * (y * z)
found:   1 = x * (y * i(x * y)) from (x * y) * z = x * (y * z), b * i(b) = 1
found:   b * (a * i(b * a)) = 1 from x * i(x) = 1, (b * a) * c = b * (a * c)
found:   1 * z = b * (i(b) * z) from (x * y) * z = x * (y * z), b * i(b) = 1
found:   (b * (a * c)) * z = (b * a) * (c * z) from (x * y) * z = x * (y * z), (b * a) * c = b * (a * c)
added:   b * (i(b) * z) = z
found:   c * (a * (i(c * a) * z)) = z from b * (i(b) * z) = z, (c * a) * d = c * (a * d)
found:   b * 1 = i(i(b)) from b * (i(b) * z) = z, a * i(a) = 1
found:   b * c = i(i(b)) * c from b * (i(b) * z) = z, a * (i(a) * c) = c
found:   d * z = e * ((i(e) * d) * z) from (x * y) * z = x * (y * z), e * (i(e) * d) = d
found:   d = x * (y * (i(x * y) * d)) from (x * y) * z = x

#### RL Systems

Similarly, if we have a *right neutral element* and a *left inverse* the resulting structure need not be a group. Systems of this kind are called *RL system*.

In [61]:
!cat Examples/rl-system.eqn || type Examples\rl-system.eqn

      x * 1 = x;
   i(x) * x = 1;
(x * y) * z = x * (y * z);

In [62]:
%%time
Rules = knuth_bendix_algorithm('Examples/rl-system.eqn')

given:   x * 1 = x
given:   (x * y) * z = x * (y * z)
given:   i(x) * x = 1
added:   x * 1 = x
added:   i(x) * x = 1
found:   1 = i(1) from x * 1 = x, i(a) * a = 1
found:   i(1) = 1 from i(x) * x = 1, a * 1 = a
added:   i(1) = 1
added:   (x * y) * z = x * (y * z)
found:   1 * z = i(b) * (b * z) from (x * y) * z = x * (y * z), i(b) * b = 1
found:   b * z = b * (1 * z) from (x * y) * z = x * (y * z), b * 1 = b
found:   (b * (a * c)) * z = (b * a) * (c * z) from (x * y) * z = x * (y * z), (b * a) * c = b * (a * c)
added:   b * (1 * z) = b * z
found:   c * (a * (1 * z)) = (c * a) * z from b * (1 * z) = b * z, (c * a) * d = c * (a * d)
found:   1 = i(1 * z) * z from b * (1 * z) = b * z, i(a) * a = 1
found:   i(1 * d) * d = 1 from i(x) * x = 1, c * (1 * d) = c * d
found:   (x * y) * d = x * (y * (1 * d)) from (x * y) * z = x * (y * z), e * (1 * d) = e * d
found:   (e * d) * z = e * ((1 * d) * z) from (x * y) * z = x * (y * z), e * (1 * d) = e * d
added:   i(1 * d) * d = 1
found:   i(1 * (1 *

### Central Groupoids

A structure $\mathcal{G} = \langle G, *\rangle$ is a *central groupoid* iff
1. $G$ is a a non-empty set.
2. $*: G \times G \rightarrow G$,
3. The following equation holds for all $x,y,z \in G$:
   $$ (x * y) * (y * z) = y $$
   
Central Groupoids have been defined by Trevor Adams in his paper [Products of Points—Some Simple Algebras and Their Identities](https://www.semanticscholar.org/paper/Products-of-Points—Some-Simple-Algebras-and-Their-Evans/0d9885e3cb4398234735759119a3dd7c67e9541d) and are also discussed by Donald E. Knuth in his paper [notes on Central Groupoids](https://www.sciencedirect.com/science/article/pii/S0021980070800321).

In [ ]:
!cat Examples/central-groupoid.eqn || type Examples\central-groupoid.eqn

In [ ]:
%%time
Rules = knuth_bendix_algorithm('Examples/central-groupoid.eqn')

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

### Quasigroups

A structure $\mathcal{G} = \langle G, *, /, \backslash \rangle$ is a [quasigroup](https://en.wikipedia.org/wiki/Quasigroup) iff
1. $G$ is a non-empty set.
2. $*: G \times G \rightarrow G$, 

   where $*$ is called the *multiplication* of $\mathcal{G}$.
3. $/: G \times G \rightarrow G$, 

   where $/$ is called the *left division* of $\mathcal{G}$.   
4. $\backslash: G \times G \rightarrow G$, 

   where $\backslash$ is called the *right division* of $\mathcal{G}$.   
5. The following equations hold for all $x,y \in G$:
   * $x * (x \backslash y) = y$, 
   * $(x / y) * y = x$,
   * $x \backslash (x * y) = y$,
   * $(x * y) / y = x$.

In [ ]:
!cat Examples/quasigroups.eqn || type Examples\quasigroups.eqn

In [ ]:
%%time
Rules = knuth_bendix_algorithm('Examples/quasigroups.eqn')

### Quasigroups with Idempotence

A *quasigroup with idempotence* is a quasigroup that additionally satisfies the identity $x * x = x$.  Therefore, a structure $\mathcal{G} = \langle G, *, /, \backslash \rangle$ is a *quasigroup with idempotence* iff
1. $G$ is a set.
2. $*: G \times G \rightarrow G$, 

   where $*$ is called the *multiplication* of $\mathcal{G}$.
3. $/: G \times G \rightarrow G$, 

   where $/$ is called the *left division* of $\mathcal{G}$.   
4. $\backslash: G \times G \rightarrow G$, 

   where $\backslash$ is called the *right division* of $\mathcal{G}$.   
5. The following equations hold for all $x,y \in G$:
   * $x * (x \backslash y) = y$, 
   * $(x / y) * y = x$,
   * $x \backslash (x * y) = y$,
   * $(x * y) / y = x$,
   * $x * x = x$.

In [ ]:
!cat Examples/quasigroup-idempotence.eqn || type Examples\quasigroup-idempotence.eqn

In [ ]:
%%time
Rules = knuth_bendix_algorithm('Examples/quasigroup-idempotence.eqn')

### Quasigroups with Unipotence

A *quasigroup with idempotence* is a quasigroup that additionally satisfies the identity $x * x = 1$
where $1$ is a constant symbol. Therefore, a structure $\mathcal{G} = \langle G, 1, *, /, \backslash \rangle$ is a *quasigroup with idempotence* iff
1. $G$ is a set.
2. $1 \in G$.
2. $*: G \times G \rightarrow G$, 

   where $*$ is called the *multiplication* of $\mathcal{G}$.
3. $/: G \times G \rightarrow G$, 

   where $/$ is called the *left division* of $\mathcal{G}$.   
4. $\backslash: G \times G \rightarrow G$, 

   where $\backslash$ is called the *right division* of $\mathcal{G}$.   
5. The following equations hold for all $x,y \in G$:
   * $x * (x \backslash y) = y$, 
   * $(x / y) * y = x$,
   * $x \backslash (x * y) = y$,
   * $(x * y) / y = x$,
   * $x * x = 1$.

In [ ]:
!cat Examples/quasigroup-unipotence.eqn || type Examples\quasigroup-unipotence.eqn

In [ ]:
%%time
Rules = knuth_bendix_algorithm('Examples/quasigroup-unipotence.eqn')

### Loops

A *loop* is a quasigroup that additionally has an identity element.  Therefore, a structure $\mathcal{G} = \langle G, 1, *, /, \backslash \rangle$ is a [loop](https://en.wikipedia.org/wiki/Quasigroup#Loops) iff
1. $G$ is a set.
2. $1 \in G$.
2. $*: G \times G \rightarrow G$, 

   where $*$ is called the *multiplication* of $\mathcal{G}$.
3. $/: G \times G \rightarrow G$, 

   where $/$ is called the *left division* of $\mathcal{G}$.   
4. $\backslash: G \times G \rightarrow G$, 

   where $\backslash$ is called the *right division* of $\mathcal{G}$.   
5. The following equations hold for all $x,y \in G$:
   * $1 * x = x$,
   * $x * 1 = x$,
   * $x * (x \backslash y) = y$, 
   * $(x / y) * y = x$,
   * $x \backslash (x * y) = y$,
   * $(x * y) / y = x$.

In [ ]:
!cat Examples/loops.eqn || type Examples\loops.eqn

In [ ]:
%%time
Rules = knuth_bendix_algorithm('Examples/loops.eqn')